# Lab 2. Solar Panel Instructions

## Introduction

In this notebook we show you how to create your second sub-agent on Amazon Bedrock Agents.

This agent contains instructions on how to install and to do maintenance on Solar Panels, where customers can ask the agent to return these information from an [Amazon Bedrock Knowledge Base](https://aws.amazon.com/bedrock/knowledge-bases/).

To equip foundation models (FMs) with up-to-date and proprietary information, organizations use Retrieval Augmented Generation (RAG), a technique that fetches data from company data sources and enriches the prompt to provide more relevant and accurate responses. 

Amazon Bedrock Knowledge Bases is Bedrock's fully managed capability that helps you implement the entire RAG workflow from ingestion to retrieval and prompt augmentation without having to build custom integrations to data sources and manage data flows.

In the context of our agent, if the answer to a question is not present on the Knowledge Base, customers can ask agent to create a support ticket, to get a human in the loop to help with their questions.

The following represents the piece of architecture that will be built on this module.

![Architecture](img/solar_panel_agent.png)

## Setup

Make sure that your boto3 version is the latest

If not, return no [notebook 1](../1-energy-forecast/1_forecasting_agent.ipynb) and run Setup block again.

In [1]:
!pip freeze | grep boto3

boto3==1.37.6


## Creating Agent

On this section we declare global variables that will be act as helpers during entire notebook and we will start to create out second agent.

In [2]:
import boto3
import os

sts_client = boto3.client('sts')
session = boto3.session.Session()

account_id = sts_client.get_caller_identity()["Account"]
region = session.region_name
account_id_suffix = account_id[:3]
agent_suffix = f"{region}-{account_id_suffix}"
s3_client = boto3.client('s3', region)
bedrock_client = boto3.client('bedrock-runtime', region)

agent_foundation_model = [
    'anthropic.claude-3-haiku-20240307-v1:0',
    'anthropic.claude-3-sonnet-20240229-v1:0',
    'anthropic.claude-3-5-sonnet-20240620-v1:0'
]

In [3]:
solar_agent_name = f"solar-p-{agent_suffix}"

solar_lambda_name = f"fn-solar-p-{agent_suffix}"

solar_agent_role_name = f'AmazonBedrockExecutionRoleForAgents_{solar_agent_name}'

dynamodb_table = f"{solar_agent_name}-table"
dynamodb_pk = "customer_id"
dynamodb_sk = "ticket_id"

dynamoDB_args = [dynamodb_table, dynamodb_pk, dynamodb_sk]

knowledge_base_name = f'{solar_agent_name}-kb'
suffix = f"{region}-{account_id}"

knowledge_base_description = "KB containing solar panel instructions for installation and maintenance"
bucket_name = f'solar-p-kb-{suffix}'

### Importing helper functions

On following section, we're adding `bedrock_agent_helper.py` and `knowledge_base_helper` on Python path, so the files can be recognized and their functionalities can be invoked.

Now, you're going to import from helper classes `bedrock_agent_helper.py` and `knowledge_base_helper.py`.
 
Those files contain helper classes totally focused on make labs experience smoothly. 

All interactions with Bedrock will be handled by these classes.

Following are methods that you're going to invoke on this lab:

On `agents.py`:
- `create_agent`: Create a new agent and respective IAM roles
- `add_action_group_with_lambda`: Create a lambda function and add it as an action group for a previous created agent
- `create_agent_alias`: Create an alias for this agent
- `invoke`: Execute agent

On `knowledge_bases.py`:
- `create_or_retrieve_knowledge_base`: Create Knowledge Base on Amazon Bedrock if it doesn't exist or get info about previous created.
- `synchronize_data`: Read files on S3, convert text info into vectors and add that information on Vector Database.

In [4]:
import sys

sys.path.insert(0, ".")
sys.path.insert(1, "..")

from utils.bedrock_agent_helper import (
    AgentsForAmazonBedrock
)
from utils.knowledge_base_helper import (
    KnowledgeBasesForAmazonBedrock
)
agents = AgentsForAmazonBedrock()
kb = KnowledgeBasesForAmazonBedrock()

## Create and Load Knowledge Base

On this section, you're going to create a Amazon Bedrock Knowledge Base and ingest data on it.

In the next steps we will generate the data used to populate the knowledge base. It will be composed of instructions on how to handle a solar panel

**This creation process takes several minutes.**

In [5]:
%%time
kb_id, ds_id = kb.create_or_retrieve_knowledge_base(
    knowledge_base_name,
    knowledge_base_description,
    bucket_name
)

print(f"Knowledge Base ID: {kb_id}")
print(f"Data Source ID: {ds_id}")

Creating KB solar-p-us-west-2-790-kb
Step 1 - Creating or retrieving solar-p-kb-us-west-2-790231119814 S3 bucket for Knowledge Base documents
Bucket solar-p-kb-us-west-2-790231119814 already exists - retrieving it!
Step 2 - Creating Knowledge Base Execution Role (AmazonBedrockExecutionRoleForKnowledgeBase_784) and Policies
Step 3 - Creating OSS encryption, network and data access policies
Step 4 - Creating OSS Collection (this step takes a couple of minutes to complete)
{ 'ResponseMetadata': { 'HTTPHeaders': { 'connection': 'keep-alive',
                                         'content-length': '320',
                                         'content-type': 'application/x-amz-json-1.0',
                                         'date': 'Wed, 05 Mar 2025 23:59:40 '
                                                 'GMT',
                                         'x-amzn-requestid': '11f816a6-a0c0-4de0-bef2-b346f6fe8da2'},
                        'HTTPStatusCode': 200,
                    

### Create Synthetic Data to Load on S3

Instead of get data elsewhere, you're going to generate data, using a LLM on Amazon Bedrock.
This fake data that will be generated, will be uploaded into a S3 bucket and then added into an Amazon Bedrock Knowledge Base.

In [6]:
path = "kb_documents"
# Check whether the specified path exists or not
isExist = os.path.exists(path)
if not isExist:
   # Create a n ew directory if it does not exist
   os.makedirs(path)
   print("The {} directory was created!".format(path))
else:
   print("The {} directory already exists!".format(path))

The kb_documents directory already exists!


Creating helper methods to invoke LLM on Bedrock and to write a local file using Python

In [7]:
def invoke_bedrock_generate_energy_files(prompt):
    message_list = []

    initial_message = {
        "role": "user",
        "content": [
            { "text": prompt } 
        ],
    }

    message_list.append(initial_message)

    response = bedrock_client.converse(
        modelId=agent_foundation_model[0],
        messages=message_list,
        inferenceConfig={
            "maxTokens": 2048,
            "temperature": 0
        },
    )

    return response['output']['message']


def write_file(file_name, content):
    f = open(file_name, 'w')
    f.write(content)
    f.close

Generating one file with instructions on how to install a solar panel.

In [8]:
text_generation_energy_instructions = '''
    You will act as an expert on clean energy.
    You will generate a step-by-step on how to install a solar panel at home.
    You know the following fictional solar panel models: Sunpower X, Sunpower Y
    and Sunpower double-X. For each one of those models, provide some general
    model description and its features. Next provide a numbered list describing
    how to install each model of solar panel. Include information about how to
    ensure compliance with energy rules.

    Answer only with the instructions and solar panel descriptions.
    Avoid answer with afirmations like: "OK, I can generate it,",
    "As an expert on clean energy, I ", or "Yes, please find following example."
    Be direct and only reply the instructions and descriptions.
'''

solar_energy_file_name = 'solar-panel-instructions.txt'

response_message = invoke_bedrock_generate_energy_files(
    text_generation_energy_instructions
)
description_and_instructions = response_message['content'][0]['text']
print(f"Generated data:\n{description_and_instructions}")
write_file(
    '{}/{}'.format(path, solar_energy_file_name),
    description_and_instructions
)

Generated data:
Sunpower X:
This solar panel model is designed for residential use, with a power output of 350 watts and an efficiency of 22%. It features a sleek, low-profile design and is made with high-quality monocrystalline silicon cells. The Sunpower X is known for its durability and long-lasting performance, with a 25-year warranty.

Sunpower Y:
The Sunpower Y is a more powerful model, with a power output of 400 watts and an efficiency of 23%. It is suitable for larger homes or those with higher energy demands. This model features advanced tracking technology, which allows it to follow the sun's movement and maximize energy production throughout the day.

Sunpower Double-X:
The Sunpower Double-X is the most advanced model in the lineup, with a power output of 450 watts and an efficiency of 24%. It is designed for homeowners who want to maximize their solar energy production. This model features a unique dual-panel design, which allows it to capture more sunlight and generate mor

### Generating data prompt
Generating another file with instructions on how to do maintenance on a solar panel.

In [9]:
text_generation_energy_instructions = f'''
    You will be act as an expert on clean energy.
    You know the following fictional solar panel models: Sunpower X, Sunpower Y
    and Sunpower double-X. Here is are some descriptions of the different
    models and how to install them:
    <description_and_instructions>
    {description_and_instructions}
    </description_and_instructions>
    Generate a step-by-step instructions on how to do maintenance on each of
    those models at a regular home. Include information about how to
    ensure consistent compliance with energy rules.
    Just answer in a numbered list.
'''

solar_energy_file_name = 'solar-panel-maintenance.txt'

response_message = invoke_bedrock_generate_energy_files(
    text_generation_energy_instructions
)
print(f"Generated data:\n{response_message['content'][0]['text']}")

write_file(
    '{}/{}'.format(path, solar_energy_file_name),
    response_message['content'][0]['text']
)


Generated data:
Maintenance Instructions for Sunpower Solar Panel Models:

Sunpower X:

1. Regularly inspect the solar panels for any visible damage, such as cracks, discoloration, or debris buildup.
2. Clean the solar panels using a soft-bristled brush and a mild, pH-neutral cleaning solution. Avoid using abrasive cleaners or high-pressure water, as they can damage the panels.
3. Check the electrical connections and ensure they are tight and free from corrosion. Tighten any loose connections and replace any damaged wiring or connectors.
4. Inspect the inverter and other system components for proper functioning. Check for any error codes or warning lights, and address any issues according to the manufacturer's recommendations.
5. Monitor the system's energy production and compare it to the expected output. Significant deviations may indicate a problem that requires further investigation or maintenance.
6. Ensure that the system is in compliance with local energy regulations by regularl

### Uploading data to S3
Uploading generated files into S3 Bucket.

In [10]:
def upload_directory(path, bucket_name):
    for root,dirs,files in os.walk(path):
        for file in files:
            file_to_upload = os.path.join(root,file)
            print(f"uploading file {file_to_upload} to {bucket_name}")
            s3_client.upload_file(file_to_upload,bucket_name,file)

### Synchronizing Knowledge Base
Now that the data is available in the s3 bucket, let's synchronize it to our knowledge base

In [11]:
upload_directory("kb_documents", bucket_name)

# sync knowledge base
kb.synchronize_data(kb_id, ds_id)

uploading file kb_documents/solar-panel-instructions.txt to solar-p-kb-us-west-2-790231119814
uploading file kb_documents/solar-panel-maintenance.txt to solar-p-kb-us-west-2-790231119814
{ 'dataSourceId': 'HDBGUXHVCJ',
  'ingestionJobId': 'WH7FX5BYZB',
  'knowledgeBaseId': 'BCT5W5W7PJ',
  'startedAt': datetime.datetime(2025, 3, 6, 0, 3, 26, 666091, tzinfo=tzlocal()),
  'statistics': { 'numberOfDocumentsDeleted': 0,
                  'numberOfDocumentsFailed': 0,
                  'numberOfDocumentsScanned': 0,
                  'numberOfMetadataDocumentsModified': 0,
                  'numberOfMetadataDocumentsScanned': 0,
                  'numberOfModifiedDocumentsIndexed': 0,
                  'numberOfNewDocumentsIndexed': 0},
  'status': 'STARTING',
  'updatedAt': datetime.datetime(2025, 3, 6, 0, 3, 26, 666091, tzinfo=tzlocal())}
{ 'dataSourceId': 'HDBGUXHVCJ',
  'ingestionJobId': 'WH7FX5BYZB',
  'knowledgeBaseId': 'BCT5W5W7PJ',
  'startedAt': datetime.datetime(2025, 3, 6, 0, 3, 2

## Creating Agent

Create the Solar Panel agent that will have a Knowledge Base and also a Lambda action group to handle exception workflow (when information is not found on KB).

For this agent we will use the following instructions:
```
You are a Solar Energy Assistant that helps customers with solar panel installation and maintenance guidance.

Your capabilities include:
1. Providing installation instructions
2. Providing maintenance procedures
3. Troubleshooting common issues
4. Creating support tickets for specialist assistance

Core behaviors:
1. Always use available information before asking customers for additional details
2. Maintain a professional yet approachable tone
3. Provide clear, direct answers
4. Present technical information in an easy-to-understand manner
5. NEVER invent information not available in your knowledge base

Support ticket protocol:
- Only generate tickets for specialist-level issues
- Respond exclusively with case ID when creating tickets
- Decline providing specialist advice beyond your scope

Response style:
- Be helpful and solution-oriented
- Use clear, practical language
- Focus on actionable guidance
- Maintain natural conversation flow
- Be concise yet informative
- Do not add extra information not required by the user
```
We will also connect a knowledge base with the information about solar panels

```
Access the knowledge base when customers ask about to install and maintain solar panels
```
And we will make the following tool available to the agent:
- `open_ticket`: to open new support tickets
- `get_ticket_status`: to get the current status of an existing ticket

In [12]:
kb_info = kb.get_kb(kb_id)
kb_arn = kb_info['knowledgeBase']['knowledgeBaseArn']

In [13]:
kb_config = {
    'kb_id': kb_id,
    'kb_instruction': 'Access the knowledge base when customers ask about to install and maintain solar panels'
}

In [14]:
agent_instruction = """You are a Solar Energy Assistant that helps customers with solar panel installation and maintenance guidance.

Your capabilities include:
1. Providing installation instructions
2. Offering maintenance procedures
3. Troubleshooting common issues
4. Creating support tickets for specialist assistance

Core behaviors:
1. Always use available information before asking customers for additional details
2. Maintain a professional yet approachable tone
3. Provide clear, direct answers
4. Present technical information in an easy-to-understand manner

Support ticket protocol:
- Only generate tickets for specialist-level issues
- Respond exclusively with case ID when creating tickets
- Decline providing specialist advice beyond your scope

Response style:
- Be helpful and solution-oriented
- Use clear, practical language
- Focus on actionable guidance
- Maintain natural conversation flow
- Be concise yet informative
- Do not add extra information not required by the user"""

agent_description = """You are a solar energy helper bot. 
    You can retrieve information on how to install and do maintenance on solar panels"""

solar_agent = agents.create_agent(
    solar_agent_name,
    agent_description,
    agent_instruction,
    agent_foundation_model,
    kb_arns=[kb_arn]
)
solar_agent

('08JDUGZSS7',
 'TSTALIASID',
 'arn:aws:bedrock:us-west-2:790231119814:agent-alias/08JDUGZSS7/TSTALIASID')

In [15]:
agents.associate_kb_with_agent(
    solar_agent[0], kb_config['kb_instruction'], kb_config['kb_id']
)

Waiting for agent status to change. Current status CREATING
Agent id 08JDUGZSS7 current status: NOT_PREPARED


### Creating Action Group

On this session, we're going create an action group to handle support tickets and associate it with our agent. To do so, we will first create a Lambda function code to fulfill the execution of the agent's actions Next we will define the actions available actions that an agent can take using function details. Similar to the previous agent, you can also define the actions available using OpenAPI Schema.

#### Creating Lambda function
First let's create the lambda function

In [16]:
%%writefile solar_energy.py
import os
import json
import uuid
import boto3

from boto3.dynamodb.conditions import Key, Attr

dynamodb_resource = boto3.resource('dynamodb')
dynamodb_table = os.getenv('dynamodb_table')
dynamodb_pk = os.getenv('dynamodb_pk')
dynamodb_sk = os.getenv('dynamodb_sk')

def get_named_parameter(event, name):
    try:
        return next(item for item in event['parameters'] if item['name'] == name)['value']
    except:
        return None
    
def populate_function_response(event, response_body):
    return {'response': {'actionGroup': event['actionGroup'], 'function': event['function'],
                'functionResponse': {'responseBody': {'TEXT': {'body': str(response_body)}}}}}

def put_dynamodb(table_name, item):
    table = dynamodb_resource.Table(table_name)
    resp = table.put_item(Item=item)
    return resp

def read_dynamodb(table_name: str, 
                   pk_field: str,
                   pk_value: str,
                   sk_field: str=None, 
                   sk_value: str=None):
    try:
        table = dynamodb_resource.Table(table_name)
        # Create expression
        if sk_value:
            key_expression = Key(pk_field).eq(pk_value) & Key(sk_field).begins_with(sk_value)
        else:
            key_expression = Key(pk_field).eq(pk_value)

        query_data = table.query(
            KeyConditionExpression=key_expression
        )
        
        return query_data['Items']
    except Exception:
        print(f'Error querying table: {table_name}.')

def open_ticket(customer_id, msg):
    ticket_id = str(uuid.uuid1())
    item = {
        'ticket_id': ticket_id,
        'customer_id': customer_id,
        'description': msg,
        'status': 'created'
    }
    resp = put_dynamodb(dynamodb_table, item)
    print(resp)
    return "Thanks for contact customer {}! Your support case was generated with ID: {}".format(
        customer_id, ticket_id
    )

def get_ticket_status(customer_id,
                      ticket_id: str=None):
    return read_dynamodb(dynamodb_table, 
                         dynamodb_pk,
                         customer_id,
                         dynamodb_sk,
                         ticket_id)

def lambda_handler(event, context):
    print(event)
    
    # name of the function that should be invoked
    function = event.get('function', '')

    # parameters to invoke function with
    parameters = event.get('parameters', [])
    customer_id = get_named_parameter(event, "customer_id")

    if function == 'open_ticket':
        msg = get_named_parameter(event, "msg")
        result = open_ticket(customer_id, msg)
    elif function == 'get_ticket_status':
        ticket_id = get_named_parameter(event, "ticket_id")
        result = get_ticket_status(customer_id, ticket_id)
    else:
        result = f"Error, function '{function}' not recognized"

    response = populate_function_response(event, result)
    print(response)
    return response

Overwriting solar_energy.py


### Defining available actions
Now it's time to define the actions that can be taken by the agent

In [17]:
functions_def =[
    {
        "name": "open_ticket",
        "description": """Create a ticket to get help with information related with solar panel or clean energy""",
        "parameters": {
            "customer_id": {
                "description": "Unique customer identifier",
                "required": True,
                "type": "string"
            },
            "msg": {
                "description": "The reason why customer is opening a ticket",
                "required": True,
                "type": "string"
            }
        }
    },
    {
        "name": "get_ticket_status",
        "description": """get the status of an existing ticket""",
        "parameters": {
            "customer_id": {
                "description": "Unique customer identifier",
                "required": True,
                "type": "string"
            },
            "ticket_id": {
                "description": "Unique ticket identifier",
                "required": False,
                "type": "string"
            }
        }
    }
]

#### Associating action group to agent
Finally, we can associate a new action group with our previously created agent

In [18]:
resp = agents.add_action_group_with_lambda(
    agent_name=solar_agent_name,
    lambda_function_name=solar_lambda_name,
    source_code_file="solar_energy.py",
    agent_functions=functions_def,
    agent_action_group_name="solar_energy_actions",
    agent_action_group_description="Function to open an energy ticket for a user or get status from an opened ticket",
    dynamo_args=dynamoDB_args
)

Table solar-p-us-west-2-790-table already exists, skipping table creation step


ResourceConflictException: An error occurred (ResourceConflictException) when calling the CreateFunction operation: Function already exist: fn-solar-p-us-west-2-790

## Testing Agent

Now, let's run some tests on the agent we just created to make sure it's working. To do so we will use our test alias: `TSTALIASID` which allows you to invoke a draft version of your agent

### Testing maintainance question
First let's ask a question related to maintaining an existing solar panel

In [ ]:
## Testing Agent


In [ ]:
%%time
response = agents.invoke(
    "Can I get details on my ticket? My customer id is 1 get my ticket id from our previous conversation {}".format(ticket), 
    solar_agent[0], enable_trace=True
)
print("====================")
print(response)

In [ ]:
solar_agent_alias_id, solar_agent_alias_arn = agents.create_agent_alias(
    solar_agent[0], 'v1'
)
solar_agent_id = solar_agent[0]

Store environment variables to be used on next notebooks.

In [ ]:
solar_agent_arn = agents.get_agent_arn_by_name(solar_agent_name)
solar_kb = knowledge_base_name
solar_dynamodb = dynamodb_table

%store solar_agent_arn
%store solar_agent_alias_arn
%store solar_agent_alias_id
%store solar_lambda_name
%store solar_agent_name
%store solar_agent_id
%store solar_kb
%store solar_dynamodb

In [ ]:
solar_agent_arn, solar_agent_alias_arn, solar_agent_alias_id

## Next Steps
Congratulations! We've now created a solar panel agent. Next we will create our peak loader manager agent